In [27]:
demand = []
x = []
y = []

with open("vrp_16_3_1") as file:
    next(file)
    for line in file:
        values = line.split()
        if len(values) == 3:
            d, x_coord, y_coord = values
            demand.append(int(d))
            x.append(float(x_coord))
            y.append(float(y_coord))

print("Demand:", demand)
print("X coordinates:", x)
print("Y coordinates:", y)


Demand: [0, 7, 30, 16, 9, 21, 15, 19, 23, 11, 5, 19, 29, 23, 21, 10]
X coordinates: [30.0, 37.0, 49.0, 52.0, 20.0, 40.0, 21.0, 17.0, 31.0, 52.0, 51.0, 42.0, 31.0, 5.0, 12.0, 36.0]
Y coordinates: [40.0, 52.0, 49.0, 64.0, 26.0, 30.0, 47.0, 63.0, 62.0, 33.0, 21.0, 41.0, 32.0, 25.0, 42.0, 16.0]


In [28]:
def distance(x1,y1,x2,y2):
    return ((x1-x2)**2+(y1-y2)**2)**0.5

In [29]:
distance_matrix = [[0 for _ in range(len(demand))] for _ in range(len(demand))]
print(distance_matrix)
for i in range(len(demand)):
    for j in range(len(demand)):
        temp=distance(x[i],y[i],x[j],y[j])
        distance_matrix[i][j]=temp
print(distance_matrix)



[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0.0, 13.892443989449804, 21.02379604162864, 32.55764119219941, 17.204650534085253, 14.142135623730951, 11.40175425099138, 26.419689627245813, 22.02271554554524, 23.08679276123039, 28.31960451701259

In [30]:
def get_node(link):
    link = link[1:]
    link = link[:-1]
    nodes = link.split(',')
    return [int(nodes[0]), int(nodes[1])]

In [31]:
def interior(node, route):
    try:
        i = route.index(node)
        # adjacent to depot, not interior
        if i == 0 or i == (len(route) - 1):
            label = False
        else:
            label = True
    except:
        label = False
    
    return label

In [32]:
def merge(route0, route1, link):
    if route0.index(link[0]) != (len(route0) - 1):
        route0.reverse()
    
    if route1.index(link[1]) != 0:
        route1.reverse()
        
    return route0 + route1

In [33]:
def sum_cap(route):
    sum_cap = 0
    for node in route:
        sum_cap += demand[node]
    return sum_cap

In [34]:
def which_route(link, routes):
    # assume nodes are not in any route
    node_sel = list()
    i_route = [-1, -1]
    count_in = 0
    
    for route in routes:
        for node in link:
            try:
                route.index(node)
                i_route[count_in] = routes.index(route)
                node_sel.append(node)
                count_in += 1
            except:
                pass
                
    if i_route[0] == i_route[1]:
        overlap = 1
    else:
        overlap = 0
        
    return node_sel, count_in, i_route, overlap

In [35]:
# create empty routes
routes = list()

# if there is any remaining customer to be served
remaining = True

# define capacity of the vehicle
cap = 23

# record steps
step = 0

# get a list of nodes, excluding the depot
node_list = list(range(len(demand)))
node_list.remove(0)

# run through each link in the saving list
for link in sv.index:
    step += 1
    if remaining:

        print('step ', step, ':')

        link = get_node(link)
        node_sel, num_in, i_route, overlap = which_route(link, routes)

        # condition a. Either, neither i nor j have already been assigned to a route, 
        # ...in which case a new route is initiated including both i and j.
        if num_in == 0:
            if sum_cap(link) <= cap:
                routes.append(link)
                node_list.remove(link[0])
                node_list.remove(link[1])
                print('\t','Link ', link, ' fulfills criteria a), so it is created as a new route')
            else:
                print('\t','Though Link ', link, ' fulfills criteria a), it exceeds maximum load, so skip this link.')

        # condition b. Or, exactly one of the two nodes (i or j) has already been included 
        # ...in an existing route and that point is not interior to that route 
        # ...(a point is interior to a route if it is not adjacent to the depot D in the order of traversal of nodes), 
        # ...in which case the link (i, j) is added to that same route.    
        elif num_in == 1:
            n_sel = node_sel[0]
            i_rt = i_route[0]
            position = routes[i_rt].index(n_sel)
            link_temp = link.copy()
            link_temp.remove(n_sel)
            node = link_temp[0]

            cond1 = (not interior(n_sel, routes[i_rt]))
            cond2 = (sum_cap(routes[i_rt] + [node]) <= cap)

            if cond1:
                if cond2:
                    print('\t','Link ', link, ' fulfills criteria b), so a new node is added to route ', routes[i_rt], '.')
                    if position == 0:
                        routes[i_rt].insert(0, node)
                    else:
                        routes[i_rt].append(node)
                    node_list.remove(node)
                else:
                    print('\t','Though Link ', link, ' fulfills criteria b), it exceeds maximum load, so skip this link.')
                    continue
            else:
                print('\t','For Link ', link, ', node ', n_sel, ' is interior to route ', routes[i_rt], ', so skip this link')
                continue

        # condition c. Or, both i and j have already been included in two different existing routes 
        # ...and neither point is interior to its route, in which case the two routes are merged.        
        else:
            if overlap == 0:
                cond1 = (not interior(node_sel[0], routes[i_route[0]]))
                cond2 = (not interior(node_sel[1], routes[i_route[1]]))
                cond3 = (sum_cap(routes[i_route[0]] + routes[i_route[1]]) <= cap)

                if cond1 and cond2:
                    if cond3:
                        route_temp = merge(routes[i_route[0]], routes[i_route[1]], node_sel)
                        temp1 = routes[i_route[0]]
                        temp2 = routes[i_route[1]]
                        routes.remove(temp1)
                        routes.remove(temp2)
                        routes.append(route_temp)
                        try:
                            node_list.remove(link[0])
                            node_list.remove(link[1])
                        except:
                            #print('\t', f"Node {link[0]} or {link[1]} has been removed in a previous step.")
                            pass
                        print('\t','Link ', link, ' fulfills criteria c), so route ', temp1, ' and route ', temp2, ' are merged')
                    else:
                        print('\t','Though Link ', link, ' fulfills criteria c), it exceeds maximum load, so skip this link.')
                        continue
                else:
                    print('\t','For link ', link, ', Two nodes are found in two different routes, but not all the nodes fulfill interior requirement, so skip this link')
                    continue
            else:
                print('\t','Link ', link, ' is already included in the routes')
                continue

        for route in routes: 
            print('\t','route: ', route, ' with load ', sum_cap(route))
    else:
        print('-------')
        print('All nodes are included in the routes, algorithm closed')
        break

    remaining = bool(len(node_list) > 0)

# check if any node is left, assign to a unique route
for node_o in node_list:
    routes.append([node_o])

# add depot to the routes
for route in routes:
    route.insert(0,0)
    route.append(0)

print('------')
print('Routes found are: ')

routes

NameError: name 'sv' is not defined